In [2]:
from sort import *
import numpy as np
import timeit



c:\ProgramData\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\ProgramData\miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
sample_path = r"E:\HD_VNese_map\main_src\exe_window\det_logs\2024-03-20_15-25-51_____TAI6_AI_bd_TQ_HD.txt"
videopath = r"C:\Users\Administrator\Videos\TAI6_AI_bd_TQ_HD.mxf"

In [4]:
# read the sample file
with open(sample_path, "r") as file:
    data = file.readlines()
stride = 5

In [5]:
for d in data:
    print(d)

INFO:root:280;china;1048.0,2.375,1280.0,473.5;0.45;1280,720

INFO:root:305;china;415.25,289.5,841.0,602.5;0.8;1280,720

INFO:root:305;vietnam;622.0,550.0,738.0,706.0;0.73;1280,720

INFO:root:310;china;385.5,286.5,853.5,621.5;0.8;1280,720

INFO:root:310;vietnam;618.0,568.0,740.0,718.0;0.75;1280,720

INFO:root:315;china;347.75,257.0,894.0,665.0;0.74;1280,720

INFO:root:315;vietnam;605.0,596.0,763.0,720.0;0.6;1280,720

INFO:root:320;china;304.0,231.5,924.0,685.0;0.72;1280,720

INFO:root:320;vietnam;599.0,617.0,758.0,718.0;0.48;1280,720

INFO:root:325;china;306.0,229.5,932.0,697.0;0.73;1280,720

INFO:root:325;vietnam;601.0,623.0,767.0,719.0;0.45;1280,720

INFO:root:330;china;293.0,215.5,934.0,702.0;0.74;1280,720

INFO:root:330;vietnam;604.0,632.0,778.0,718.0;0.58;1280,720

INFO:root:335;china;364.75,208.125,1009.0,688.0;0.73;1280,720

INFO:root:335;vietnam;703.0,615.0,883.0,719.0;0.55;1280,720

INFO:root:340;china;362.0,217.0,1016.0,689.0;0.71;1280,720

INFO:root:340;vietnam;711.0,615.0,90

In [9]:
data_dict = {}
width, heihgt = 0, 0
for line in data:
    line = line[10:]
    line = line.split(";")
    frame_number = int(line[0])
    width, height = line[4].split(",")
    height = height.replace("\n", "")
    width, height = int(width), int(height)
    class_name = line[1]
    bbox = [int(float(i)) for i in line[2].split(",")]
    score = float(line[3])
    detection = bbox + [score]
    detection = np.array(detection)
    detection = np.expand_dims(detection, axis=0)
    if frame_number not in list(data_dict.keys()):
        data_dict[frame_number] = detection
    else:
        data_dict[frame_number] = np.vstack((data_dict[frame_number], detection))

In [10]:
tracker = Sort(max_age=2, min_hits=1, iou_threshold=0.1)
tracker.reset() # set the counter to 0 every new video
id_counter_dict = {}
data_dict_with_id = {}
start_time = timeit.default_timer()
for i in range(0, max(list(data_dict.keys())), stride):
    if i not in list(data_dict.keys()):
        detection = np.array([[0, 0, 0, 0, 0]])
        state = tracker.update(detection)
        continue
    # check if data_dict[i] is tuple or not
    if type(data_dict[i]) == np.ndarray:
        try:
            detection = data_dict[i] #.reshape(1, -1)
            #start_time = timeit.default_timer()
            state = tracker.update(detection)
            #print("time: ", timeit.default_timer() - start_time)
        except:
            print("error")
            print(data_dict[i])
        #state = tracker.update(data_dict[i])
        for res in state:
            res = res.astype(int)
            x1, y1, x2, y2, id = res
            
            if id not in id_counter_dict:
                id_counter_dict[id] = []
            id_counter_dict[id].append(i)
            
            print("frame: ", i, "id: ", id, "x1: ", x1, "y1: ", y1, "x2: ", x2, "y2: ", y2)
            if i not in list(data_dict_with_id.keys()):
                data_dict_with_id[i] = np.array([x1, y1, x2, y2, id])
                data_dict_with_id[i] = np.expand_dims(data_dict_with_id[i], axis=0)
            else:
                data_dict_with_id[i] = np.vstack((data_dict_with_id[i], [x1, y1, x2, y2, id]))
    else:
        print(data_dict[i])
        print("error")

print("time: ", timeit.default_timer() - start_time)
    

e:\HD_VNese_map\main_src\exe_window\sort.py:77: RuntimeWarning: invalid value encountered in divide
  r = w / float(h)


frame:  310 id:  63 x1:  619 y1:  566 x2:  738 y2:  719
frame:  310 id:  62 x1:  386 y1:  284 x2:  851 y2:  622
frame:  315 id:  63 x1:  615 y1:  587 x2:  752 y2:  728
frame:  315 id:  62 x1:  348 y1:  262 x2:  891 y2:  660
frame:  320 id:  63 x1:  608 y1:  607 x2:  750 y2:  729
frame:  320 id:  62 x1:  308 y1:  234 x2:  921 y2:  684
frame:  325 id:  63 x1:  609 y1:  618 x2:  756 y2:  728
frame:  325 id:  62 x1:  295 y1:  225 x2:  939 y2:  700
frame:  330 id:  63 x1:  612 y1:  627 x2:  765 y2:  726
frame:  330 id:  62 x1:  281 y1:  213 x2:  946 y2:  707
frame:  335 id:  63 x1:  682 y1:  621 x2:  845 y2:  723
frame:  335 id:  62 x1:  328 y1:  201 x2:  1002 y2:  702
frame:  340 id:  63 x1:  713 y1:  618 x2:  886 y2:  722
frame:  340 id:  62 x1:  344 y1:  201 x2:  1025 y2:  703
frame:  345 id:  63 x1:  714 y1:  615 x2:  895 y2:  721
frame:  345 id:  62 x1:  349 y1:  197 x2:  1029 y2:  699
frame:  350 id:  63 x1:  712 y1:  615 x2:  894 y2:  721
frame:  350 id:  62 x1:  350 y1:  195 x2:  10

In [16]:

import cv2
video = cv2.VideoCapture(videopath)
# export the video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (width, height))
print("width: ", width, "height: ", height)
while True:
    ret, frame = video.read()
    if ret:
        frame_number = int(video.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_number in list(data_dict_with_id.keys()):
            bboxs = data_dict_with_id[frame_number][:, :4]
            ids = data_dict_with_id[frame_number][:, 4]
            for i, bbox in enumerate(bboxs):
                x1, y1, x2, y2 = bbox
                id = ids[i]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, str(id), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        #cv2.imshow("frame", frame)
        out.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        cv2.waitKey(1)
    else:
        break
video.release()
out.release()

cv2.destroyAllWindows()


In [25]:
for i in list(id_counter_dict.keys()):
    print(i, (id_counter_dict[i]))

4 [10, 15, 20, 25, 30, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
3 [10, 15, 20, 25, 30, 35, 45]
2 [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110]
5 [25, 30, 35, 40, 45, 50, 55, 60, 65, 75, 80, 85, 90, 95, 100]
58 [380, 385]
59 [390, 395, 400, 405, 410, 415, 420, 425, 430, 435, 440]
60 [405, 420]
128 [785, 790, 800, 805, 810, 815, 820, 825]
135 [855, 860, 875, 880, 885, 890, 895, 900, 905, 910, 915, 920, 925]
134 [855, 860, 865, 870, 875, 880, 885, 890, 895, 900, 905, 910, 915, 920, 925]
173 [1120]


In [15]:
data_dict.keys()

dict_keys([5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 375, 380, 385, 390, 395, 400, 405, 410, 415, 420, 425, 430, 435, 440, 780, 785, 790, 800, 805, 810, 815, 820, 825, 850, 855, 860, 865, 870, 875, 880, 885, 890, 895, 900, 905, 910, 915, 920, 925, 1115, 1120, 1130])

In [2]:
img_folder = "E:\HD_VNese_map\main_src\samples"

In [5]:
import os
# convert extention of the images to jpg
for filename in os.listdir(img_folder):
    ext = filename.split(".")[-1]
    if ext != "jpg":
        os.rename(os.path.join(img_folder, filename), os.path.join(img_folder, filename.split(".")[0] + ".jpg"))

In [6]:
import cv2
import numpy as np
orange = np.uint8([[[0, 0, 255]]])
hsv_orange = cv2.cvtColor(orange, cv2.COLOR_BGR2HSV)
print(hsv_orange)

[[[  0 255 255]]]


In [15]:
import cv2 
import numpy as np

def orange_color_identifier(img, threshold=10):
    """
    This function takes an image as input and returns True if the image contains a significant amount of orange color,
    and False otherwise. The threshold parameter is used to determine the minimum percentage of orange pixels required
    to return True. The default threshold is 10%.
    (adjust the lower and upper bounds for the orange color to match the specific shade of orange in the image.)
    """


    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the orange color
    lower_orange = np.array([0, 30, 30])
    upper_orange = np.array([30, 255, 255])

    # Create a mask that selects only the orange pixels
    mask = cv2.inRange(hsv_image, lower_orange, upper_orange)
    cv2.imshow("mask", mask)
    cv2.waitKey(0)
    # Count the number of orange pixels
    orange_pixels = np.sum(mask > 0)

    # Calculate the percentage of orange pixels
    total_pixels = img.shape[0] * img.shape[1]
    percentage_orange = (orange_pixels / total_pixels) * 100
    #print("percentage_orange", percentage_orange)   
    print("percentage_orange", percentage_orange)
    if percentage_orange < threshold:
        return False
    else:
        return True


In [16]:
import matplotlib.pyplot as plt
import os
img_folder = "E:\HD_VNese_map\main_src\samples"
# Load the image
for filename in os.listdir(img_folder):
    img = cv2.imread(os.path.join(img_folder, filename))
   
    if img is None:
        continue
    if orange_color_identifier(img, 5):
        print(filename, "contains orange color")
    else:
        print(filename, "does not contain orange color")
    
plt.show()

percentage_orange 34.25
1.jpg contains orange color
percentage_orange 63.51656249999999
10.jpg contains orange color
percentage_orange 22.78648926902669
2.jpg contains orange color
percentage_orange 11.293073442394709
3.jpg contains orange color
percentage_orange 26.32636591719002
4.jpg contains orange color
percentage_orange 0.0
5.jpg does not contain orange color
percentage_orange 20.709239587136558
6.jpg contains orange color
percentage_orange 0.596875
8.jpg does not contain orange color
